In [1]:
import json

from doml_mc.model.doml_model import parse_doml_model
from doml_mc.z3.z3_doml_model import Z3DOMLModel

In [2]:
with open("POSIDONIA.doml") as jsonf:
    doc = json.load(jsonf)

doml_model = parse_doml_model(doc)
z3_model = Z3DOMLModel(doml_model)


In [3]:
z3_model.solver.check()

sat

In [4]:
z3_model.solver.model()

[infra_node_type = [prodevelop/edi/procesos:latest ->
                    infrastructure_Container,
                    prodevelop/edimer/procesos:latest ->
                    infrastructure_Container,
                    else -> infrastructure_VirtualMachine],
 consumes = [(EstacionEDI, HTTP, ProcesosEDI-WS) -> True,
             (ProcesosEDI-WS, EDIMERTEL HTTP, EDIMERTEL) ->
             True,
             else -> False],
 network_proto = [else -> TCP/IP],
 network_first_addr = [Subnet2 -> 174326272,
                       Subnet3 -> 174326528,
                       else -> 174326016],
 network_last_addr = [Subnet2 -> 174326527,
                      Subnet3 -> 174326783,
                      else -> 174326271],
 infra_group_type = [GestautASG ->
                     infrastructure_AutoScalingGroup,
                     ElasticSearchASG ->
                     infrastructure_AutoScalingGroup,
                     EdiASG ->
                     infrastructure_AutoScalingGroup,
                     else -> infrastructure_SecurityGroup],
 app_type = [ORACLE -> application_SaaSDBMS,
             else -> application_SoftwarePackage]]

In [5]:
print(z3_model.solver.sexpr())

(declare-datatypes ((ApplicationInterface 0)) (((HTTP) (|EDIMERTEL HTTP|))))
(declare-datatypes ((InfrastructureNetwork 0)) (((Subnet1) (Subnet2) (Subnet3))))
(declare-datatypes ((InfrastructureNodeType 0)) (((infrastructure_Container) (infrastructure_VirtualMachine))))
(declare-datatypes ((ApplicationComponent 0)) (((EstacionEDI) (ProcesosEDI-WS) (EDIMERTEL) (VALEDIENGINE) (EDIMER-SERVICESTEDI) (POSIDONIA-PCS) (POSIDONIA-ESTEDI) (POSIDONIA-GESTAUT) (VALEDI) (PCS-SERVICES) (ESTEDI-SERVICES) (GESTAUT-SERVICES) (ORACLE))))
(declare-datatypes ((InfrastructureGroup 0)) (((GestautASG) (ElasticSearchASG) (EdiASG) (BalearesDBSG) (BalearesSG) (BalearesELBSG) (CheckmkSG))))
(declare-datatypes ((ApplicationType 0)) (((application_SaaSDBMS) (application_SoftwarePackage))))
(declare-datatypes ((InfrastructureNode 0)) (((|Plataforma EDI|) (EDIMER) (DATABASE) (PIDEWEB) (|ELASTIC SEARCH|) (PMS) (|prodevelop/edi/procesos:latest|) (|prodevelop/edimer/procesos:latest|))))
(declare-datatypes ((NetworkPro